# Joins columns from benchmarks and jobsarchive tables and outputs them for ML analysis

In [1]:
import pickle
import re
import datetime
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import scan

import numpy as np
import pandas as pd

es = Elasticsearch([{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

#start_date = datetime.datetime(2017, 4, 7)
start_date = datetime.datetime(2017, 5, 1)
#curre_date = datetime.datetime(2017, 4, 30, 23, 59, 59)
curre_date = datetime.datetime.utcnow()
days_of_data = (curre_date - start_date).days

benchmark_indices = ['benchmarks-2017.05'] #'benchmarks-2017.04',

In [2]:
# get job archive indices
indices = es.cat.indices(index="jobs_archive_20*", h="index", request_timeout=600).split('\n')
indices = sorted(indices)
indices = [x for x in indices if x != '']
#for i in indices:
#    print(i)

selected_indices=[] 
for d in range(days_of_data+1):
    td = start_date + datetime.timedelta(d)
    ind = 'jobs_archive_'+td.strftime('%Y-%m-%d')
    if ind in indices:
        selected_indices.append(ind)

job_indices = ','.join(selected_indices)
print(job_indices)

jobs_archive_2017-05-01,jobs_archive_2017-05-02,jobs_archive_2017-05-03,jobs_archive_2017-05-04,jobs_archive_2017-05-05,jobs_archive_2017-05-06,jobs_archive_2017-05-07,jobs_archive_2017-05-08,jobs_archive_2017-05-09,jobs_archive_2017-05-10,jobs_archive_2017-05-11,jobs_archive_2017-05-12,jobs_archive_2017-05-13,jobs_archive_2017-05-14,jobs_archive_2017-05-15,jobs_archive_2017-05-16,jobs_archive_2017-05-17,jobs_archive_2017-05-18,jobs_archive_2017-05-19,jobs_archive_2017-05-20,jobs_archive_2017-05-21,jobs_archive_2017-05-22,jobs_archive_2017-05-23,jobs_archive_2017-05-24,jobs_archive_2017-05-25,jobs_archive_2017-05-26,jobs_archive_2017-05-27,jobs_archive_2017-05-28,jobs_archive_2017-05-29,jobs_archive_2017-05-30,jobs_archive_2017-05-31,jobs_archive_2017-06-01,jobs_archive_2017-06-02,jobs_archive_2017-06-03,jobs_archive_2017-06-04,jobs_archive_2017-06-05,jobs_archive_2017-06-06,jobs_archive_2017-06-07,jobs_archive_2017-06-08,jobs_archive_2017-06-09,jobs_archive_2017-06-10,jobs_archive_201

In [3]:
benchmarks_query={
    "size": 0,
    "query":{
       "match_all": {}
    }
}

res = scan(es, query=benchmarks_query, index=benchmark_indices, scroll='5m', timeout="5m", size=1000)

data=[]
benchmarksPerSite={}
count = 0
for r in res:
    pr=r['_source']['profiles']
    mt=r['_source']['metadata']
    site=mt['ATLASSite']
    if site not in benchmarksPerSite:
        benchmarksPerSite[site]=0
    benchmarksPerSite[site]+=1
    if 'fastBmk' not in pr: 
        fastBmk = 0
    else:
        fastBmk = pr['fastBmk']['value']
    if 'whetstone' not in pr:
        ws = 0
    else:
        ws = pr['whetstone']['score']
    doc=[mt['PanDAID'],mt['bogomips'],mt['ip'],site,mt['mp_num'],mt['cpuname'],mt['meminfo'],fastBmk,ws]
    data.append(doc)
    if not count%10000:
        print(count)
#     if count>100: break
    count+=1

print(len(data))
    
benchmark=pd.DataFrame(data,columns=['pandaid','bogomips','ip','site','mpnum','cpuname','meminfo','fastBmk','whetstone'])
del data

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
94600


In [4]:
#print(benchmark.dtypes)
benchmark.head()
pids=set(benchmark['pandaid'].tolist())
print('total benchmarks:', len(pids))


total benchmarks: 94597


In [5]:
benchmark.to_csv('benchmark.csv',index=False)


In [6]:
# select only production jobs as only these can potentially have benchmark done. 
# prodsourcelabel = prod_test, rc_test, managed, install
job_query = {
    "size": 0,
    "_source": ["cpuconsumptiontime", "wall_time", "hs06","nevents","inputfilebytes","prodsourcelabel","processingtype","computingsite"],
    'query':{
            'bool':{
                   'must':[
                        { "term": {"jobstatus": "finished" } },
                        { "term": {"prodsourcelabel": "managed" } }
                    ],
                   'should':[
                      # { "term": {"prodsourcelabel": "rc_test" } },
                      # { "term": {"prodsourcelabel": "prod_test" } },
                       { "term": {"prodsourcelabel": "managed" } }
                   ]
            }
    }
                
}

data=[]
scroll = scan(client=es, index=job_indices, query=job_query, scroll='5m', timeout="5m", size=10000)
count = 0
counttype={'managed':0,'rc_test':0,'prod_test':0,'install':0}
jobsPerSite={}
for res in scroll:
    count += 1
    #print(res)
    #if count>3: break
    
    r=res['_source']
    
    site=r['computingsite']
    if site not in jobsPerSite:
        jobsPerSite[site]=0
    jobsPerSite[site]+=1
    
    if not count%100000: 
        print(count, ' selected:', counttype)
        #print(data)
    
    pid=int(res['_id'])
    if pid not in pids: continue
    
    counttype[r['prodsourcelabel']]+=1
    cpu=r['cpuconsumptiontime']
    ifb=r['inputfilebytes']
    wall=r['wall_time']
    nevents=r['nevents']
    if wall>0: cpueff = cpu/wall
    if nevents>0:
        wallPerEvent = wall/nevents
        cpuPerEvent = cpu/nevents
    else:
        wallPerEvent = 0
        cpuPerEvent =0
    doc=[pid,cpu,wall,r['hs06'],r['processingtype'],nevents, cpueff, wallPerEvent, cpuPerEvent,ifb]
    data.append(doc)

job=pd.DataFrame(data,columns=['pandaid','cputime','walltime','hs06','processingtype','nevents','cpueff', 'wallPerEvent', 'cpuPerEvent','inputsize'])
del data

100000  selected: {'install': 0, 'managed': 563, 'rc_test': 0, 'prod_test': 0}
200000  selected: {'install': 0, 'managed': 1107, 'rc_test': 0, 'prod_test': 0}
300000  selected: {'install': 0, 'managed': 1650, 'rc_test': 0, 'prod_test': 0}
400000  selected: {'install': 0, 'managed': 2163, 'rc_test': 0, 'prod_test': 0}
500000  selected: {'install': 0, 'managed': 2676, 'rc_test': 0, 'prod_test': 0}
600000  selected: {'install': 0, 'managed': 3217, 'rc_test': 0, 'prod_test': 0}
700000  selected: {'install': 0, 'managed': 3809, 'rc_test': 0, 'prod_test': 0}
800000  selected: {'install': 0, 'managed': 4355, 'rc_test': 0, 'prod_test': 0}
900000  selected: {'install': 0, 'managed': 4861, 'rc_test': 0, 'prod_test': 0}
1000000  selected: {'install': 0, 'managed': 5425, 'rc_test': 0, 'prod_test': 0}
1100000  selected: {'install': 0, 'managed': 6001, 'rc_test': 0, 'prod_test': 0}
1200000  selected: {'install': 0, 'managed': 6545, 'rc_test': 0, 'prod_test': 0}
1300000  selected: {'install': 0, 'man

In [7]:
job.head()

,pandaid,cputime,walltime,hs06,processingtype,nevents,cpueff,wallPerEvent,cpuPerEvent,inputsize
0,3362958467,29847,30068,9,evgen,200,0.992650,150.340000,149.235000,21500648
1,3389673630,298653,41337,77,simul,1000,7.224835,41.337000,298.653000,21524372
2,3361062104,12586,3052,122,reprocessing,1071,4.123853,2.849673,11.751634,1133004708
3,3395197447,1278,1685,10,recon,223,0.758457,7.556054,5.730942,1464209160
4,3395213853,1137,1499,10,recon,241,0.758506,6.219917,4.717842,2323509696


#### store all the jobs together

In [8]:
job.to_csv('job.csv',index=False)

#### store jobs split per processingtype

In [9]:
UniqueProcessingTypes = job.processingtype.unique()
print(UniqueProcessingTypes)

#create a dict for all the dataframes to be filled later
ProcessingType = {elem : pd.DataFrame for elem in UniqueProcessingTypes}

#filling up data frames
for key in ProcessingType.keys():
    job[:][job.processingtype == key].to_csv(key+'.csv',index=False)

['evgen' 'simul' 'reprocessing' 'recon' 'pile' 'deriv' 'skim' 'merge'
 'pmerge' 'eventIndex' 'overlay']


In [10]:
#full = pd.merge(benchmark, job, on='pandaid')
#full.head()
#full.to_csv('benchmark_job.csv',index=False)

In [11]:
#for ps in benchmarksPerSite:
#    print(ps, benchmarksPerSite[ps])

for ps in jobsPerSite:
    if ps not in benchmarksPerSite and jobsPerSite[ps]>1000: 
        print(ps, jobsPerSite[ps])#,'       <<<<<<<<<<<<<')
    #else:
        #print(ps, jobsPerSite[ps],benchmarksPerSite[ps])

CSCS-LCG2-HPC 6994
HPC2N_MCORE 25352
RAL-LCG2_VAC 22047
CONNECT_ODYSSEY 2413
MWT2_VHIMEM_MCORE 2102
NERSC_Edison_2 30057
CONNECT_FRESNOSTATE 1106
HPC2N 24897
CA-VICTORIA-WESTGRID-T2_MCORE 65282
LRZ-LMU_MUC1_MCORE 114632
SiGNET-NSC_MCORE 38041
RAL-AZURE_VAC 10932
BNL_PROD_CONTR 69930
UKI-LT2-IC-HEP_MCORE_SL7 1183
RRC-KI-HPC2 11801
CONNECT_ICCHEPT3 1844
CA-VICTORIA-WESTGRID-T2 6698
BNL_LOCAL 1356
LRZ-LMU_MUC_MCORE1 6535
SiGNET 15017
CPPM_MCORE 20723
CONNECT_KARST 4033
CONNECT_ES_JETSTREAM 1756
OU_OSCER_ATLAS 7737
CONNECT_ES_KARST 1265
OU_OSCER_ATLAS_OPP 8493
BNL_PROD_MCORE 492633
ORNL_Titan_MCORE 1166596
CONNECT_UIUC 3070
OU_OSCER_ATLAS_MCORE 5140
CONNECT_ES_ICC 2364
Anselm_MCORE 18944
CSCS-LCG2-HPC_MCORE 4827
CONNECT_JETSTREAM 11311
BNL_PROD 268939
CONNECT_ES_ODYSSEY_MCORE 5547
SiGNET_MCORE 64071
BNL_ATLAS_2 1273
CONNECT_ES_ODYSSEY 21925


In [12]:
print (jobsPerSite)
#print (benchmarksPerSite)

{'SARA-MATRIX': 56655, 'RAL-LCG2_VHIMEM': 404, 'UNIBE-LHEP_CLOUD_MCORE': 20, 'UIO_MCORE_LOPRI': 3398, 'ALCF_Cooley': 40, 'MWT2_VHIMEM': 2109, 'Taiwan-LCG2': 54118, 'UKI-NORTHGRID-MAN-HEP_MCORE': 41367, 'INFN-BOLOGNA-T3': 10630, 'CSCS-LCG2-HPC': 6994, 'PSNC': 36087, 'HPC2N_MCORE': 25352, 'LRZ-LMU': 22933, 'GRIF-LPNHE_MCORE': 35411, 'LPC': 36782, 'IN2P3-CC': 66211, 'Taiwan-LCG2-HPC_MCORE': 13554, 'Lucille_MCORE': 1577, 'CONNECT_ES_UIUC': 657, 'pic_HTCondor-mcore': 3464, 'UKI-LT2-IC-HEP_MCORE': 26425, 'RAL-LCG2_VAC': 22047, 'MWT2_SL6': 286946, 'SWT2_CPB_MCORE': 169121, 'CERN-PROD_HI': 143811, 'ARNES': 22808, 'TRIUMF_HIMEM': 1263, 'UKI-SOUTHGRID-RALPP_MCORE': 14472, 'ITEP_PROD': 2409, 'LAPP_MCORE': 43028, 'CYFRONET-LCG2': 1335, 'RAL-LCG2-ECHO': 32873, 'CONNECT_ODYSSEY': 2413, 'TRIUMF_MCORE_LOMEM': 159806, 'INFN-COSENZA-RECAS': 26452, 'BEIJING': 10395, 'MPPMU-DRACO_MCORE': 5767, 'MWT2_VHIMEM_MCORE': 2102, 'UNIBE-LHEP_MCORE': 11844, 'ROMANIA07': 25095, 'NERSC_Edison_2': 30057, 'CONNECT_FRESN